# Exercise 3
Ole Gunnar Hovland and Alexander Hatle

In [1]:
%matplotlib inline
import numpy as np
import time
import matplotlib.pyplot as plt
newparams = {'figure.figsize': (8.0, 4.0), 'axes.grid': True,
             'lines.markersize': 8, 'lines.linewidth': 2,
             'font.size': 14}
from matplotlib import cm
plt.rcParams.update(newparams)

In [12]:
def tridiag(v, d, w, N):
    # Help function 
    # Returns a tridiagonal matrix A=tridiag(v, d, w) of dimension N x N.
    if len(v) + len(d) + len(w) > 3:
        A = np.diag(v, -1) + np.diag(d) + np.diag(w, 1)
    else:
        e = np.ones(N)        # array [1,1,...,1] of length N
        A = v*np.diag(e[1:],-1) + d * np.diag(e) + w * np.diag(e[1:],1)
    return A

In [ ]:
def crank(g, params, BC, M=10, N=100, R = 1, T=0.5):
    # Input: 
    #       g: initial function for t=0
    #       M, N: number of grid intervals in the x- and t directions
    #       T: end of integration
    # Output: 
    #       x, t: the gridpoints in the x- and t- directions 
    #       U: An array with the numerical solution.

    

    # Set the stepsizes
    h = R/M     # Stepsize in space
    k = T/N     # Stepsize in time

    # Parameters
    sigma = params[0]
    r = params[1]
    c = params[2]
    K = params[3]

    # Helping parameters
    alpha = sigma**2 * k / (4 * h**2)
    beta = r * k / (4*h)
    gamma = c * k / 2

    # Print the stepsizes, and r=k/h^2.
    print('h={:.4f}, k={:.4f}'.format(h,k))

    U = np.zeros((M+1,N+1))    # Array to store the solution, boundaries included.
    x = np.linspace(0,R,M+1)   # Gridpoints on the x-axis
    t = np.linspace(0,T,N+1)   # Gridpoints on the t-axis
    U[:,0] = g(x)              # Initial values, endpoints included
    
    # Constructing A
    lowdiag = -alpha * x[2:-1]**2 - beta * x[2:-1]         # M-2
    diag =  1 + alpha * x[1:]**2 + gamma                   # M-1
    updiag = -alpha * x[1:-2]**2 + beta * x[1:-2]         # M-2
    A = tridiag(lowdiag, diag, updiag, M-1)

    # Constructing B
    lowdiag = alpha * x[2:-1]**2 + beta * x[2:-1]  
    diag = 1 - alpha * x[1:]**2 - gamma 
    updiag = alpha * x[2:-1]**2 + beta * x[2:-1] 
    B = tridiag(lowdiag, diag, updiag, M-1)

    # Constructing p and q
    p = np.zeros(M-1)
    p[-1] = (-alpha * x[-1]**2 - beta * x[-1]) * U[-1, 0]
    q = -p

    # Boundary functions
    if BC == 'EP':
        def bndry(t):
            return K * np.exp(-c * t)
    elif BC == 'binary':
        def bndry(t):
            return 0

    # Main loop 
    for n in range(N):
        U[1:-1, n+1] = A.dot(U[1:-1,n])  # [1:-1] -> excluding endpoints
        # Boundary
        U[0, n+1] = bndry(t[n+1])
    return x, t, U